<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_stunning_flyer_in_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a stunning flyer in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [16]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


For the next snippet of code, we're going to need these imports. They form the basis of any workflow (with borb) in which you create a PDF.

In [17]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout

These are the steps to creating a PDF document using `borb`:

- Create an empty `Document`
- Create an empty `Page`
- Append the `Page` to the `Document`
- Set a `PageLayout` to handle the flow of content (we're using a `SingleColumnLayout` here)
- Add content (not shown here)
- Write the PDF to disk (not shown here)

In [18]:
# create empty Document
pdf = Document()

# create empty Page
page = Page()

# add Page to Document
pdf.add_page(page)

# create PageLayout
layout: PageLayout = SingleColumnLayout(page)

We'd like to add some geometric artwork to our flyer in the upper right corner. We're going to write a separate method to do that. Then we can later re-use this method (for instance on every `Page` in the `Document`).

In [19]:
# new imports
from borb.pdf.canvas.layout.shape.connected_shape import ConnectedShape
from decimal import Decimal
from borb.pdf.canvas.color.color import HexColor, X11Color
from borb.pdf.canvas.geometry.rectangle import Rectangle
from borb.pdf.page.page_size import PageSize
import typing
import random

def add_gray_artwork_to_upper_right_corner(page: Page) -> None:
  """
  This method will add a gray artwork of squares and triangles in the upper right corner
  of the given Page
  """

  # define a list of gray colors
  grays: typing.List[HexColor] = [HexColor("A9A9A9"), 
                                HexColor("D3D3D3"), 
                                HexColor("DCDCDC"), 
                                HexColor("E0E0E0"),
                                HexColor("E8E8E8"),
                                HexColor("F0F0F0")]

  # we're going to use the size of the page later on,
  # so perhaps it's a good idea to retrieve it now                                
  ps: typing.Tuple[Decimal, Decimal] = PageSize.A4_PORTRAIT.value

  # now we'll write N triangles in the upper right corner
  # we'll later fill the remaining space with squares
  N: int = 4
  M: Decimal = Decimal(32)
  for i in range(0, N):
    x: Decimal = ps[0] - N * M + i * M
    y: Decimal = ps[1] - (i+1) * M
    rg: HexColor = random.choice(grays)
    ConnectedShape(points=[(x + M,y), (x + M, y + M), (x, y + M)], stroke_color=rg, fill_color=rg).paint(page, Rectangle(x, y, M, M))

  # now we can fill up the remaining space with squares    
  for i in range(0, N-1):
    for j in range(0, N-1):
      if j > i:
        continue
      x: Decimal = ps[0] - (N-1) * M + i * M
      y: Decimal = ps[1] - (j+1) * M
      rg: HexColor = random.choice(grays)
      ConnectedShape(points=[(x, y), (x + M, y), (x + M, y + M), (x, y + M)], stroke_color=rg, fill_color=rg).paint(page, Rectangle(x, y, M, M))


Now that we've defined this method, we can call it in the main body of our script to add the artwork to the PDF.

In [20]:
# now we can call this method in the main method
add_gray_artwork_to_upper_right_corner(page)

Next we're going to add our company contact details, so people know where to reach us:

In [21]:
# new imports
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.layout.image.barcode import Barcode, BarcodeType
from borb.pdf.canvas.layout.layout_element import LayoutElement
from borb.pdf.canvas.layout.table.flexible_column_width_table import FlexibleColumnWidthTable
from borb.pdf.canvas.layout.annotation.remote_go_to_annotation import RemoteGoToAnnotation

# contact information
layout.add(Paragraph("Your Company", font_color=HexColor("#6d64e8"), font_size=Decimal(20)))

# We're going to add a qr code that links to our website.
# Later, we're going to add a remote go-to annotation 
# (that's just PDF talk for "if you click the qr code, it will take you to our website")
# in order to be able to do that, we need its coordinates.
qr_code: LayoutElement = Barcode(
    data="https://www.borbpdf.com",
    width=Decimal(64),
    height=Decimal(64),
    type=BarcodeType.QR,
)

# now we can add this content to the table
layout.add(
    FlexibleColumnWidthTable(number_of_columns=2, number_of_rows=1)
    .add(qr_code)
    .add(
      Paragraph(
          """
          500 South Buena Vista Street
          Burbank CA
          91521-0991 USA
          """,
          padding_top=Decimal(12),
          respect_newlines_in_text=True,
          font_color=HexColor("#666666"),
          font_size=Decimal(10),
      )
    )
    .no_borders()
)

# let's add the remote go-to annotation
page.add_annotation(RemoteGoToAnnotation(
  qr_code.get_previous_paint_box(), uri="https://www.borbpdf.com"
))


{<borb.io.read.types.Name at 0x7f5f64b6c310>: <borb.io.read.types.Name at 0x7f5f64b6c690>,
 <borb.io.read.types.Name at 0x7f5f64b374d0>: [Decimal('0'),
  Decimal('0'),
  Decimal('595'),
  Decimal('842')],
 <borb.io.read.types.Name at 0x7f5f64bc5c10>: {<borb.io.read.types.Name at 0x7f5f64d41310>: Decimal('1'),
  <borb.io.read.types.Name at 0x7f5f658a1d10>: [{...}],
  <borb.io.read.types.Name at 0x7f5f64aa4690>: <borb.io.read.types.Name at 0x7f5f64bcdfd0>},
 <borb.io.read.types.Name at 0x7f5f64aa45d0>: {<borb.io.read.types.Name at 0x7f5f6498d810>: b'q 0.909804 0.909804 0.909804 RG  0.909804 0.909804 0.909804 rg 1.000000 w 499.000000 810.000000 m  499.000000 842.000000 l  467.000000 842.000000 l  B  Qq 0.909804 0.909804 0.909804 RG  0.909804 0.909804 0.909804 rg 1.000000 w 531.000000 778.000000 m  531.000000 810.000000 l  499.000000 810.000000 l  B  Qq 0.662745 0.662745 0.662745 RG  0.662745 0.662745 0.662745 rg 1.000000 w 563.000000 746.000000 m  563.000000 778.000000 l  531.000000 778.0

Now we can add a few titles and subtitles and some promotional blurb;

In [22]:
# title
layout.add(Paragraph("Productbrochure", 
                     font_color=HexColor("#283592"), 
                     font_size=Decimal(34)))

# subtitle
layout.add(Paragraph("September 4th, 2021", 
                     font_color=HexColor("#e01b84"), 
                     font_size=Decimal(11)))

layout.add(Paragraph("Product Overview",
                     font_color=HexColor("000000"),
                     font_size=Decimal(21)))

layout.add(Paragraph("""
                    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
                    Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
                    Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
                    Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
                    """))

Images make things more visually interesting. Let's add an `Image` with some core product features next to it;

In [23]:
# new imports
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable
from borb.pdf.canvas.layout.table.table import TableCell
from borb.pdf.canvas.layout.image.image import Image
from borb.pdf.canvas.layout.list.unordered_list import UnorderedList

# table with image and key features
layout.add(
  FixedColumnWidthTable(number_of_rows=2, number_of_columns=2, column_widths=[Decimal(0.3), Decimal(0.7)])
  .add(
    TableCell(
      Image("https://www.att.com/catalog/en/skus/images/apple-iphone%2012-purple-450x350.png", width=Decimal(128), height=Decimal(128)),
      row_span=2,
    )
  )
  .add(Paragraph("Key Features", font_color=HexColor("e01b84"), font="Helvetica-Bold", padding_bottom=Decimal(10)))            
  .add(
    UnorderedList()
      .add(Paragraph("Nam aliquet ex eget felis lobortis aliquet sit amet ut risus."))
      .add(Paragraph("Maecenas sit amet odio ut erat tincidunt consectetur accumsan ut nunc."))
      .add(Paragraph("Phasellus eget magna et justo malesuada fringilla."))
      .add(Paragraph("Maecenas vitae dui ac nisi aliquam malesuada in consequat sapien."))
    )
  .no_borders()
)

Let's add a footer to the bottom of the page. We're going to put this in a separate method (so that we could call it later on, if we ever need to apply it to other pages in the PDF).

In [24]:
# new imports
from borb.pdf.canvas.line_art.line_art_factory import LineArtFactory

def add_colored_artwork_to_bottom_right_corner(page: Page) -> None:
  """
  This method will add a blue/purple artwork of lines and squares to the bottom right corner
  of the given Page
  """
  ps: typing.Tuple[Decimal, Decimal] = PageSize.A4_PORTRAIT.value
  
  # square
  ConnectedShape(points=[(ps[0] - 32, 40), (ps[0], 40), (ps[0], 40 + 32), (ps[0] - 32, 40 + 32)], stroke_color=HexColor("d53067"), fill_color=HexColor("d53067")).paint(page, Rectangle(ps[0] - 32, 40, 32, 32))
  
  # square
  ConnectedShape(points=[(ps[0] - 64, 40), (ps[0] - 32, 40), (ps[0] - 32, 40 + 32), (ps[0] - 64, 40 + 32)], stroke_color=HexColor("eb3f79"), fill_color=HexColor("eb3f79")).paint(page, Rectangle(ps[0] - 64, 40, 32, 32))
  
  # triangle
  ConnectedShape(points=[(ps[0] - 96, 40), (ps[0] - 64, 40), (ps[0] - 64, 40 + 32)], stroke_color=HexColor("e01b84"), fill_color=HexColor("e01b84")).paint(page, Rectangle(ps[0] - 96, 40, 32, 32))
  
  # line
  r: Rectangle = Rectangle(Decimal(0), Decimal(32), ps[0], Decimal(8))
  ConnectedShape(points=LineArtFactory.rectangle(r), stroke_color=HexColor("283592"), fill_color=HexColor("283592")).paint(page, r)


Now we can call this method in the main body;

In [25]:
add_colored_artwork_to_bottom_right_corner(page)

Finally, we can store the PDF;

In [26]:
with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, pdf)